In [1]:
import sys
sys.path.append("/Users/alex/Downloads/stats/")

import pandas as pd
import os
from db import connection
from db_utils import DBUtils
import xlrd
from tqdm import tqdm
import datetime
import json
from glob import glob

In [2]:
values = ['UN WPP - Total population (both sexes combined) by single age, region, subregion and country, annually for 1950-2100 (thousands)',
        'UN WPP - Male population by single age, region, subregion and country, annually for 1950-2100 (thousands)',
         'UN WPP - Female population by single age, region, subregion and country, annually for 1950-2100 (thousands)',
         'UN WPP - Interpolated demographic indicators by region, subregion and country, annually for 1950-2099',
          'UN WPP - Interpolated total population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)',
          'UN WPP - Interpolated male population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)',
          'UN WPP - Interpolated female population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)',
          'UN WPP - Percentage of total population by broad age group, region, subregion and country, annually interpolated for 1950-2100',
          'UN WPP - Percentage of male total population by broad age group, region, subregion and country, annually interpolated for 1950-2100',
          'UN WPP - Percentage of female total population by broad age group, region, subregion and country, annually interpolated for 1950-2100',
          'UN WPP - Dependency ratios (total, child, old-age) for different age groups and for both sexes combined by region, subregion and country, annually interpolated for 1950-2100',
          'UN WPP - Male dependency ratios (total, child, old-age) for different age groups by region, subregion and country, annually interpolated for 1950-2100',
          'UN WPP - Female dependency ratios (total, child, old-age) for different age groups by region, subregion and country, annually interpolated for 1950-2100'
         ]

keys = ['Annual population by single age - Both Sexes. De facto population as of 1 July of the year indicated classified by single age (0, 1, 2, ..., 99, 100+). Data are presented in thousands.',
       'Annual population by single age - Male. De facto population as of 1 July of the year indicated classified by single age (0, 1, 2, ..., 99, 100+). Data are presented in thousands.',
       'Annual population by single age - Female. De facto population as of 1 July of the year indicated classified by single age (0, 1, 2, ..., 99, 100+). Data are presented in thousands.',
       'Annually interpolated demographic indicators.',
       'Annual total population (both sexes combined) by broad age groups. Data are presented in thousands.',
       'Annual male population by broad age groups. Data are presented in thousands.',
        'Annual female population by broad age groups. Data are presented in thousands.',
        'Percentage of annual total population (both sexes combined) by broad age group.',
        'Percentage of annual male total population by broad age groups.',
        'Percentage of annual female total population by broad age groups.',
        'Annual dependency ratios (total, child, old-age) for different age groups and for both sexes combined.',
        'Annual male dependency ratios (total, child, old-age) for different age groups.',
        'Annual female dependency ratios (total, child, old-age) for different age groups.'
       ]
datasets_dict = {}
for x,y in zip(keys, values):
    datasets_dict[x] = y

In [3]:
# with connection as c:
#     db = DBUtils(c)
    
#     # upsert datasets
#     dataset_name_ids = {}
#     for f in tqdm(os.listdir('sources/')):
#         if f == '.DS_Store':
#             continue
#         data = pd.read_excel('sources/'+f)
#         val = data[data.columns[0]][8]
#         index_to_remove = val.find(":")
#         res = "UN WPP - " + val[index_to_remove+2:]
#         dataset_id = db.upsert_dataset(name=res, namespace="unwpp", user_id=15)
#         dataset_name_ids[res] = dataset_id
        
#     # upsert sources
    
#     dataset_to_source_ids = {}
#     source_name = "United Nations – Population Division (2019 Revision)"
#     for addInfo, dataset_name in datasets_dict.items():
#         description = {}
#         description['dataPublishedBy'] = "United Nations, Department of Economic and Social Affairs, Population Division (2019). World Population Prospects: The 2019 Revision, DVD Edition."
#         description['dataPublisherSource'] = None
#         description['link'] = 'https://population.un.org/wpp2019/Download/Standard/Interpolated/'
#         description['retrievedDate'] = datetime.datetime.now().strftime("%d-%b-%Y")
#         description['additionalInfo'] = addInfo
        
#         source_id = db.upsert_source(name=source_name, description=json.dumps(description), dataset_id=dataset_name_ids[dataset_name])
#         dataset_to_source_ids[dataset_name_ids[dataset_name]] = source_id
    

In [4]:
# dataset_to_source_ids

# Preparing csvs

## Datasets

In [5]:
names,ids = [], []
i = 0
for f in tqdm(glob('data/*.xlsx')):
    
        data = pd.read_excel(f)
        val = data[data.columns[0]][8]
        index_to_remove = val.find(":")
        res = "UN WPP - " + val[index_to_remove+2:]
        names.append(res)
        ids.append(i)
        i+=1
datasets = pd.DataFrame()
datasets['id'] = ids
datasets['name'] = names
datasets.to_csv('datasets.csv', index=False)

100%|██████████| 13/13 [05:26<00:00, 23.69s/it]


## Sources

In [6]:
names, desc, d_ids = [], [], []

source_name = "United Nations – Population Division (2019 Revision)"
for addInfo, dataset_name in datasets_dict.items():
    description = {}
    description['dataPublishedBy'] = "United Nations, Department of Economic and Social Affairs, Population Division (2019). World Population Prospects: The 2019 Revision, DVD Edition."
    description['dataPublisherSource'] = None
    description['link'] = 'https://population.un.org/wpp2019/Download/Standard/Interpolated/'
    description['retrievedDate'] = datetime.datetime.now().strftime("%d-%b-%Y")
    description['additionalInfo'] = addInfo
    dataset_id = datasets[datasets['name'] == dataset_name]['id'].values[0]

    names.append(source_name)
    desc.append(description)
    d_ids.append(dataset_id)
    
res = pd.DataFrame()
res['name'] = names
res['description'] = desc
res['dataset_id'] = d_ids
res.to_csv("sources.csv", index=False)

## Variables

In [7]:
class DataVariables():
    
    def __init__(self):
        
        
        self.i = 0
        self.ids = []
        self.names = []
        self.units = []
        self.dataset_ids = []
        self.doc_to_unit = {
                        "UN WPP - Female population by single age, region, subregion and country, annually for 1950-2100 (thousands)": "Thousands",
                        "UN WPP - Male population by single age, region, subregion and country, annually for 1950-2100 (thousands)": "Thousands",
            "UN WPP - Total population (both sexes combined) by single age, region, subregion and country, annually for 1950-2100 (thousands)": "Thousands",
            "UN WPP - Female dependency ratios (total, child, old-age) for different age groups by region, subregion and country, annually interpolated for 1950-2100": "Female dependency ratios for different age groups",
            "UN WPP - Male dependency ratios (total, child, old-age) for different age groups by region, subregion and country, annually interpolated for 1950-2100": "Male dependency ratios for different age groups",
            "UN WPP - Dependency ratios (total, child, old-age) for different age groups and for both sexes combined by region, subregion and country, annually interpolated for 1950-2100": "Dependency ratios (both sexes combined) for different age groups",
            "UN WPP - Percentage of female total population by broad age group, region, subregion and country, annually interpolated for 1950-2100": "Percentage",
            "UN WPP - Percentage of male total population by broad age group, region, subregion and country, annually interpolated for 1950-2100": "Percentage",
            "UN WPP - Percentage of total population by broad age group, region, subregion and country, annually interpolated for 1950-2100": "Percentage",
            "UN WPP - Interpolated female population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)": "Thousands",
            "UN WPP - Interpolated male population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)": "Thousands",
            "UN WPP - Interpolated total population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)": "Thousands"
            #UN WPP - Interpolated demographic indicators by region, subregion and country, annually for 1950-2099
                   

                    }
        
        self.datasets = pd.read_csv('datasets.csv')
        self.df = pd.DataFrame()
    
        
    def get_variables(self, path, skiprows=8, prefix=None):
        
        for sh in ["ESTIMATES", "MEDIUM VARIANT"]:
            data = pd.read_excel(path, skiprows=skiprows, sheet_name=sh)

            val = data[data.columns[0]][0]
            index_to_remove = val.find(":")
            res = "UN WPP - " + val[index_to_remove+2:]

            title = data[data.columns[0]][1]
            print(title)

            for item in data.loc[7, data.columns[8]: data.columns[-1]].values:
                if prefix:
                    self.names.append(title + ": " + prefix + " - " + item)
                else:
                    self.names.append(title + ": " + item)
                self.ids.append(self.i)
                self.i+=1
                self.units.append(self.doc_to_unit[res])
                self.dataset_ids.append(self.datasets[self.datasets['name'] == res]['id'].values[0])
    
    def get_custom_variable(self, path, skiprows=8, prefix=None):
        
        column_unit = {
            8: "Thousands",
            9: "Thousands",
            10: "Thousands",
            11: "Percentage",
            12: "Years",
            13: "Years",
            14: "Years",
            15: "Thousands",
            16: "Infant deaths per 1,000 live births",
            17: "Deaths under age 5 per 1,000 live births",
            18: "Thousands",
            19: "Births per 1,000 population",
            20: "Live births per woman",
            21: "Thousands",
            22: "Per 1,000 population",
            23: "Thousands",
            24: "Percentage"
            
        
        }
        
        for sh in ["ESTIMATES", "MEDIUM VARIANT"]:
            data = pd.read_excel(path, skiprows=skiprows, sheet_name=sh)

            val = data[data.columns[0]][0]
            index_to_remove = val.find(":")
            res = "UN WPP - " + val[index_to_remove+2:]

            title = data[data.columns[0]][1]
            print(title)
            
            col_index = 8
            for item in data.loc[7, data.columns[8]: data.columns[-1]].values:
                if prefix:
                    self.names.append(title + ": " + prefix + " - " + item)
                else:
                    self.names.append(title + ": " + item)
                self.ids.append(self.i)
                self.i+=1
                self.units.append(column_unit[col_index])
                self.dataset_ids.append(self.datasets[self.datasets['name'] == res]['id'].values[0])
                col_index += 1
           
    

            
            
    def get_df(self):
        
        self.df['id'] = self.ids
        self.df['name'] = self.names
        self.df['unit'] = self.units
        self.df['dataset_id'] = self.dataset_ids
        
        return self.df
        
        
        

In [9]:
dv = DataVariables()
for f in tqdm(glob('data/*.xlsx')):
     
        if f == "data/WPP2019_INT_F03_3_POPULATION_BY_AGE_ANNUAL_FEMALE.xlsx":
            dv.get_variables(f, prefix="Female population by age (thousands)")
            
        elif f == "data/WPP2019_INT_F03_2_POPULATION_BY_AGE_ANNUAL_MALE.xlsx":
            dv.get_variables(f, prefix="Male population by age (thousands)")   
        elif f == "data/WPP2019_INT_F03_1_POPULATION_BY_AGE_ANNUAL_BOTH_SEXES.xlsx":
            dv.get_variables(f, prefix="Total population by age, both sexes combined (thousands)")
            
        elif f == "data/WPP2019_INT_F02C_3_ANNUAL_POPULATION_INDICATORS_DEPENDENCY_RATIOS_FEMALE.xlsx":
            dv.get_variables(f, prefix="Female dependency ratios for different age groups")
            
        elif f == "data/WPP2019_INT_F02C_2_ANNUAL_POPULATION_INDICATORS_DEPENDENCY_RATIOS_MALE.xlsx":
            dv.get_variables(f, prefix="Male dependency ratios for different age groups")
            
        elif f == "data/WPP2019_INT_F02C_1_ANNUAL_POPULATION_INDICATORS_DEPENDENCY_RATIOS_BOTH_SEXES.xlsx":
            dv.get_variables(f, prefix="Dependency ratios (both sexes combined) for different age groups")
            
        elif f == "data/WPP2019_INT_F02B_3_ANNUAL_POPULATION_INDICATORS_PERCENTAGE_FEMALE.xlsx":
            dv.get_variables(f, prefix="Percentage of female population by broad age group (per 100 female total population)")
            
        elif f == "data/WPP2019_INT_F02B_2_ANNUAL_POPULATION_INDICATORS_PERCENTAGE_MALE.xlsx":
            dv.get_variables(f, prefix="Percentage of male population by broad age group (per 100 male total population)")
            
        elif f == "data/WPP2019_INT_F02B_1_ANNUAL_POPULATION_INDICATORS_PERCENTAGE_BOTH_SEXES.xlsx":
            dv.get_variables(f, prefix="Percentage of total population by broad age group, both sexes combined (per 100 total population)")
            
        elif f == "data/WPP2019_INT_F02A_3_ANNUAL_POPULATION_INDICATORS_FEMALE.xlsx":
            dv.get_variables(f, prefix="Female population by broad age group (thousands)")
            
        elif f == "data/WPP2019_INT_F02A_2_ANNUAL_POPULATION_INDICATORS_MALE.xlsx":
            dv.get_variables(f, prefix="Male population by broad age group (thousands)")
            
        elif f == "data/WPP2019_INT_F02A_1_ANNUAL_POPULATION_INDICATORS_BOTH_SEXES.xlsx":
            dv.get_variables(f, prefix="Total population by broad age group, both sexes combined (thousands)")
            
        elif f == "data/WPP2019_INT_F01_ANNUAL_DEMOGRAPHIC_INDICATORS.xlsx":
            dv.get_custom_variable(f, prefix="Male population by age (thousands)")
    
        

res = dv.get_df()


  0%|          | 0/13 [00:00<?, ?it/s]

Estimates, 1950 - 2020


  8%|▊         | 1/13 [00:20<04:03, 20.31s/it]

Medium fertility variant, 2020 - 2100
Estimates, 1950 - 2020


 15%|█▌        | 2/13 [01:07<05:13, 28.50s/it]

...

Medium fertility variant, 2020 - 2100
Estimates, 1950 - 2020


 92%|█████████▏| 12/13 [08:58<00:41, 41.43s/it]

Medium fertility variant, 2020 - 2100
Estimates, 1950 - 2020


100%|██████████| 13/13 [09:41<00:00, 41.98s/it]

Medium fertility variant, 2020 - 2100


In [10]:
res.to_csv("variables.csv", index=False)

## Datapoints

In [11]:
def normalize_country(row):
        
        row['country'] = row['country'].str.replace(r'\s*[^A-Za-z\s]*$', '')
        return row


def get_variables(path, variables, skiprows=8, prefix=None, custom=False):
        
        for sh in ["ESTIMATES", "MEDIUM VARIANT"]:
            data = pd.read_excel(path, skiprows=skiprows, sheet_name=sh)

            val = data[data.columns[0]][0]
            index_to_remove = val.find(":")
            res = "UN WPP - " + val[index_to_remove+2:]

            title = data[data.columns[0]][1]
            
            index_col = 8
            
            for item in data.loc[7, data.columns[8]: data.columns[-1]].values:
                
                if prefix:
                    var_name = title + ": " + prefix + " - " + item
                else:
                    var_name = title + ": " + item
                
                var_id = variables[variables['name'] == var_name]['id'].values[0] 
                
                data2 = data.iloc[8:]
               
                data_res = pd.DataFrame()
                data_res['country'] = data2[data2.columns[2]]
                data_res['year'] = data2[data2.columns[7]]
                data_res['value'] = data2["Unnamed: %s" % str(index_col)]
                
                data_res['country'] = normalize_country(data_res)
                data_res.to_csv('datapoints/datapoints_%s.csv' % str(var_id), index=False)
                
                index_col += 1
            

In [19]:
variables = pd.read_csv("variables.csv")

for f in tqdm(glob('data/*.xlsx')):
   

    if f == "data/WPP2019_INT_F03_3_POPULATION_BY_AGE_ANNUAL_FEMALE.xlsx":
            get_variables(f,variables, prefix="Female population by age (thousands)")
            
    elif f == "data/WPP2019_INT_F03_2_POPULATION_BY_AGE_ANNUAL_MALE.xlsx":
        get_variables(f,variables, prefix="Male population by age (thousands)")   
    elif f == "data/WPP2019_INT_F03_1_POPULATION_BY_AGE_ANNUAL_BOTH_SEXES.xlsx":
        get_variables(f,variables, prefix="Total population by age, both sexes combined (thousands)")

    elif f == "data/WPP2019_INT_F02C_3_ANNUAL_POPULATION_INDICATORS_DEPENDENCY_RATIOS_FEMALE.xlsx":
        get_variables(f,variables, prefix="Female dependency ratios for different age groups")

    elif f == "data/WPP2019_INT_F02C_2_ANNUAL_POPULATION_INDICATORS_DEPENDENCY_RATIOS_MALE.xlsx":
        get_variables(f,variables, prefix="Male dependency ratios for different age groups")

    elif f == "data/WPP2019_INT_F02C_1_ANNUAL_POPULATION_INDICATORS_DEPENDENCY_RATIOS_BOTH_SEXES.xlsx":
        get_variables(f,variables, prefix="Dependency ratios (both sexes combined) for different age groups")

    elif f == "data/WPP2019_INT_F02B_3_ANNUAL_POPULATION_INDICATORS_PERCENTAGE_FEMALE.xlsx":
        get_variables(f,variables, prefix="Percentage of female population by broad age group (per 100 female total population)")

    elif f == "data/WPP2019_INT_F02B_2_ANNUAL_POPULATION_INDICATORS_PERCENTAGE_MALE.xlsx":
        get_variables(f,variables, prefix="Percentage of male population by broad age group (per 100 male total population)")

    elif f == "data/WPP2019_INT_F02B_1_ANNUAL_POPULATION_INDICATORS_PERCENTAGE_BOTH_SEXES.xlsx":
        get_variables(f,variables, prefix="Percentage of total population by broad age group, both sexes combined (per 100 total population)")

    elif f == "data/WPP2019_INT_F02A_3_ANNUAL_POPULATION_INDICATORS_FEMALE.xlsx":
        get_variables(f,variables, prefix="Female population by broad age group (thousands)")

    elif f == "data/WPP2019_INT_F02A_2_ANNUAL_POPULATION_INDICATORS_MALE.xlsx":
        get_variables(f,variables, prefix="Male population by broad age group (thousands)")

    elif f == "data/WPP2019_INT_F02A_1_ANNUAL_POPULATION_INDICATORS_BOTH_SEXES.xlsx":
        get_variables(f,variables, prefix="Total population by broad age group, both sexes combined (thousands)")

    elif f == "data/WPP2019_INT_F01_ANNUAL_DEMOGRAPHIC_INDICATORS.xlsx":
        get_variables(f,variables, prefix="Male population by age (thousands)")




  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [00:22<04:34, 22.88s/it]

 15%|█▌        | 2/13 [01:20<06:06, 33.31s/it]

 23%|██▎       | 3/13 [02:51<08:25, 50.58s/it]

 31%|███       | 4/13 [04:21<09:21, 62.40s/it]

 38%|███▊      | 5/13 [04:40<06:36, 49.54s/it]

 46%|████▌     | 6/13 [05:33<05:53, 50.55s/it]

 54%|█████▍    | 7/13 [05:55<04:11, 41.98s/it]

 62%|██████▏   | 8/13 [07:26<04:43, 56.67s/it]

 69%|██████▉   | 9/13 [08:19<03:41, 55.48s/it]

 77%|███████▋  | 10/13 [08:41<02:16, 45.41s/it]

 85%|████████▍ | 11/13 [09:34<01:35, 47.70s/it]

 92%|█████████▏| 12/13 [10:27<00:49, 49.38s/it]

100%|██████████| 13/13 [11:20<00:00, 50.34s/it]

## Get country names

In [17]:
# countries = set()

# for x in tqdm(glob('datapoints/*.csv')):
    
   
#     data = pd.read_csv(x)
#     for j in data['country'].values:
#         countries.add(j)
# res = pd.DataFrame()
# res['name'] = list(countries)
# res.to_csv("distinct_countries_standardized.csv", index=False)


100%|██████████| 356/356 [00:06<00:00, 51.32it/s]

True

## Insert db

In [20]:
with connection as c:
    db = DBUtils(c)
    
    entities = pd.read_csv("distinct_countries_standardized.csv")
    
    new_entities = entities[entities['db_entity_id'].isnull()]
    for _, entity in new_entities.iterrows():
        entity_id = entity.name
        entity_name = entity['name']
        db_entity_id = db.get_or_create_entity(entity_name)
        entities.loc[entity_id, 'db_entity_id'] = db_entity_id
    
    # upsert datasets
    dataset_name_ids = {}
    for f in tqdm(glob('data/*.xlsx')):
        
        data = pd.read_excel(f)
        val = data[data.columns[0]][8]
        index_to_remove = val.find(":")
        res = "UN WPP - " + val[index_to_remove+2:]
        dataset_id = db.upsert_dataset(name=res, namespace="unwpp", user_id=15)
        dataset_name_ids[res] = dataset_id
        
    # upsert sources
    
    dataset_to_source_ids = {}
    source_name = "United Nations – Population Division (2019 Revision)"
    for addInfo, dataset_name in tqdm(datasets_dict.items()):
        description = {}
        description['dataPublishedBy'] = "United Nations, Department of Economic and Social Affairs, Population Division (2019). World Population Prospects: The 2019 Revision, DVD Edition."
        description['dataPublisherSource'] = None
        description['link'] = 'https://population.un.org/wpp2019/Download/Standard/Interpolated/'
        description['retrievedDate'] = datetime.datetime.now().strftime("%d-%b-%Y")
        description['additionalInfo'] = addInfo
        
        source_id = db.upsert_source(name=source_name, description=json.dumps(description), dataset_id=dataset_name_ids[dataset_name])
        dataset_to_source_ids[dataset_name] = source_id
        
    
    variables = pd.read_csv('variables.csv')
    datasets = pd.read_csv('datasets.csv')
    names_to_ids = {}
    for i, row in tqdm(variables.iterrows()):
        
        dataset_name = datasets[datasets['id'] == row['dataset_id']]['name'].values[0]
        dataset_id = dataset_name_ids[dataset_name]
        source_id = dataset_to_source_ids[dataset_name]

        
        variable_id = db.upsert_variable(
                                        name=row['name'], 
                                        code=None, 
                                        unit=row['unit'], 
                                        short_unit=None, 
                                        source_id=source_id, 
                                        dataset_id=dataset_id, 
                                        description=None, 
                                        timespan='', 
                                        coverage='', 
                                        display={}
                                        )
        names_to_ids[row['name']] = variable_id
        
    #Inserting datapoints


    datapoints_files = glob("datapoints/*.csv")
    for x in tqdm(datapoints_files): 
        # to get variable is
        v_id = int(x.split("_")[1].split(".")[0])
       
        # to get variable name
        variable_name = variables[variables['id']==v_id]['name'].values[0]
       
        # to get variable id from db
        variable_id = names_to_ids[variable_name]
        data = pd.read_csv(x)

        for i, row in data.iterrows():
            entity_id = entities[entities['name'] == row['country']]['db_entity_id'].values[0]
            if not pd.notnull(row['value']):
                val = 0.0
            else:
                val = row['value']

            year = row['year']

            db.upsert_one("""
                INSERT INTO data_values
                    (value, year, entityId, variableId)
                VALUES
                    (%s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE
                    value = VALUES(value),
                    year = VALUES(year),
                    entityId = VALUES(entityId),
                    variableId = VALUES(variableId)
            """, [val, int(year), str(int(entity_id)), str(variable_id)])
    



  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [00:10<02:02, 10.23s/it]

 15%|█▌        | 2/13 [00:32<02:32, 13.82s/it]

 23%|██▎       | 3/13 [01:08<03:25, 20.54s/it]

 31%|███       | 4/13 [01:45<03:48, 25.35s/it]

 38%|███▊      | 5/13 [01:53<02:42, 20.28s/it]

 46%|████▌     | 6/13 [02:15<02:25, 20.77s/it]

 54%|█████▍    | 7/13 [02:24<01:44, 17.36s/it]

 62%|██████▏   | 8/13 [03:01<01:55, 23.04s/it]

 69%|██████▉   | 9/13 [03:22<01:30, 22.60s/it]

 77%|███████▋  | 10/13 [03:32<00:55, 18.64s/it]

 85%|████████▍ | 11/13 [03:53<00:39, 19.50s/it]

 92%|█████████▏| 12/13 [04:15<00:20, 20.12s/it]

100%|██████████| 13/13 [04:36<00:00, 20.53s/it]

  0%|          | 0/13 [00:00<?, ?it/s]

100%|██████████| 13/13 [00:00<00:00, 441.53it/s]

0it [00:00, ?it/s]

33it [00:00, 328.38it/s]

2
3




73it [00:00, 332.31it/s]

117it [00:00, 358.40it/s]

158it [00:00, 370.79it/s]

208it [00:00, 318.30it/s]

254it [00:00, 349.77it/s]

292it [00:00, 356.82it/s]

340it [00:00, 386.38it/s]

382it [00:01, 395.74it/s]

426it [00:01, 407.04it/s]

467it [00:01, 404.51it/s]

512it [00:01, 414.61it/s]

555it [00:01, 418.13it/s]

597it [00:01, 403.59it/s]

638it [00:01, 400.50it/s]

679it [00:01, 391.12it/s]

722it [00:01, 401.69it/s]

765it [00:01, 408.02it/s]

813it [00:02, 425.55it/s]

859it [00:02, 433.12it/s]

907it [00:02, 444.11it/s]

956it [00:02, 456.18it/s]

1002it [00:02, 443.39it/s]

1047it [00:02, 442.68it/s]

1092it [00:02, 444.15it/s]

1137it [00:02, 440.79it/s]

1182it [00:02, 415.06it/s]

1224it [00:03, 412.73it/s]

1267it [00:03, 417.09it/s]

1310it [00:03, 420.73it/s]

1355it [00:03, 429.01it/s]

1399it [00:03, 409.51it/s]

1444it [00:03, 408.61it/s]

  0%|          | 0/1444 [00:00<?, ?it/s]

  0%|          | 1/1444 [00:27<11:07:15, 27.74s/it]

KeyboardInterrupt: 